# Methylation-Based Cancer Detection Classifier
## A GRAIL/Galleri-inspired approach using TCGA data

This notebook walks through building a simplified version of a multi-cancer early detection
classifier from methylation data — the same core concept behind GRAIL's Galleri test.

**What we're building:**
1. A cancer detection classifier (cancer vs normal) from methylation features
2. A tissue-of-origin classifier (which cancer type) from the same features
3. A synthetic dilution experiment simulating real ctDNA tumor fractions

**Data source:** TCGA Illumina 450K methylation arrays via the GDC Data Portal

---
## 0. Setup and Dependencies

In [ ]:
# Install dependencies (uncomment if needed)
# !pip install pandas numpy scikit-learn matplotlib seaborn xgboost requests tqdm

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import os
import gzip
import io
import joblib
from tqdm import tqdm
from pathlib import Path

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    roc_curve, roc_auc_score, precision_recall_curve,
    confusion_matrix, classification_report, auc
)
from sklearn.feature_selection import SelectKBest, f_classif

import warnings
warnings.filterwarnings('ignore')

# Try importing xgboost (optional)
try:
    from xgboost import XGBClassifier
    HAS_XGB = True
    print("XGBoost available")
except ImportError:
    HAS_XGB = False
    print("XGBoost not installed -- will skip XGBoost comparisons")

# Plot style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

# Create data directory
DATA_DIR = Path('data/tcga_methylation')
DATA_DIR.mkdir(parents=True, exist_ok=True)

print("Setup complete.")

XGBoost available
Setup complete.


---
## 1. Download TCGA Methylation Data via GDC API

We'll pull Illumina 450K methylation beta values from the GDC for 3 cancer types
that are high priority for ctDNA-based diagnostics:

- **TCGA-LUAD** — Lung adenocarcinoma
- **TCGA-COAD** — Colon adenocarcinoma  
- **TCGA-PAAD** — Pancreatic adenocarcinoma

For each project, we'll download both tumor and solid tissue normal samples.

### How GDC data works
Each sample produces a single file with ~485K CpG probe IDs and their beta values.
We query the GDC API for file UUIDs, download each file, and merge into a matrix.

In [2]:
# ------------------------------------------------------------------
# GDC API helper functions
# ------------------------------------------------------------------

GDC_FILES_ENDPOINT = "https://api.gdc.cancer.gov/files"
GDC_DATA_ENDPOINT = "https://api.gdc.cancer.gov/data"


def query_gdc_methylation_files(project_id, sample_type="Primary Tumor", max_files=30):
    """
    Query GDC API for 450K methylation beta-value files.
    
    sample_type: 'Primary Tumor' or 'Solid Tissue Normal'
    """
    filters = {
        "op": "and",
        "content": [
            {"op": "=", "content": {"field": "cases.project.project_id", "value": project_id}},
            {"op": "=", "content": {"field": "data_category", "value": "DNA Methylation"}},
            {"op": "=", "content": {"field": "data_type", "value": "Methylation Beta Value"}},
            {"op": "=", "content": {"field": "platform", "value": "Illumina Human Methylation 450"}},
            {"op": "=", "content": {"field": "cases.samples.sample_type", "value": sample_type}},
        ]
    }

    params = {
        "filters": str(filters).replace("'", '"'),
        "fields": "file_id,file_name,cases.case_id,cases.project.project_id,cases.samples.sample_type",
        "format": "JSON",
        "size": str(max_files)
    }

    response = requests.get(GDC_FILES_ENDPOINT, params=params)
    data = response.json()
    
    files = []
    for hit in data.get("data", {}).get("hits", []):
        files.append({
            "file_id": hit["file_id"],
            "file_name": hit["file_name"],
            "project": project_id,
            "sample_type": sample_type
        })
    
    print(f"  Found {len(files)} {sample_type} files for {project_id}")
    return files


def download_methylation_file(file_id, cache_dir=DATA_DIR):
    """
    Download a single methylation beta-value file from GDC.
    Returns a pandas Series of beta values indexed by CpG probe ID.
    """
    cache_path = cache_dir / f"{file_id}.pkl"
    
    # Use cached version if available
    if cache_path.exists():
        return pd.read_pickle(cache_path)
    
    url = f"{GDC_DATA_ENDPOINT}/{file_id}"
    response = requests.get(url)
    
    # GDC returns tab-separated files, possibly gzipped
    try:
        content = gzip.decompress(response.content).decode('utf-8')
    except gzip.BadGzipFile:
        content = response.content.decode('utf-8')
    
    df = pd.read_csv(io.StringIO(content), sep='\t', comment='#')
    
    # GDC 450K files typically have columns:
    # 'Composite Element REF' (probe ID) and 'Beta_value'
    probe_col = [c for c in df.columns if 'composite' in c.lower() or 'probe' in c.lower()]
    beta_col = [c for c in df.columns if 'beta' in c.lower()]
    
    if probe_col and beta_col:
        series = df.set_index(probe_col[0])[beta_col[0]]
    else:
        # Fallback: assume first col is probe ID, second is beta
        series = df.set_index(df.columns[0])[df.columns[1]]
    
    series = pd.to_numeric(series, errors='coerce')
    series.name = file_id
    
    # Cache to disk
    series.to_pickle(cache_path)
    
    return series

In [3]:
# ------------------------------------------------------------------
# Query files for each cancer type
# ------------------------------------------------------------------
# Caps are ceilings — GDC returns however many samples exist.
# Approximate availability in TCGA:
#   LUAD: ~450 tumor, ~59 solid tissue normal
#   COAD: ~300 tumor, ~38 solid tissue normal
#   PAAD: ~180 tumor, ~10 solid tissue normal
# With these settings we expect ~300 tumors and ~107 normals (~407 total).
# That gives ~32 normal test samples → reliable specificity estimates to ~97%.

PROJECTS = ["TCGA-LUAD", "TCGA-COAD", "TCGA-PAAD"]
TUMOR_SAMPLES_PER_PROJECT = 100
NORMAL_SAMPLES_PER_PROJECT = 60  # normals are scarcer, especially for PAAD

all_files = []

print("Querying GDC for methylation files...\n")
for project in PROJECTS:
    tumor_files = query_gdc_methylation_files(
        project, sample_type="Primary Tumor", max_files=TUMOR_SAMPLES_PER_PROJECT
    )
    normal_files = query_gdc_methylation_files(
        project, sample_type="Solid Tissue Normal", max_files=NORMAL_SAMPLES_PER_PROJECT
    )
    all_files.extend(tumor_files)
    all_files.extend(normal_files)

file_manifest = pd.DataFrame(all_files)
print(f"\nTotal files to download: {len(file_manifest)}")
print(file_manifest.groupby(['project', 'sample_type']).size())

Querying GDC for methylation files...

  Found 100 Primary Tumor files for TCGA-LUAD
  Found 32 Solid Tissue Normal files for TCGA-LUAD
  Found 100 Primary Tumor files for TCGA-COAD
  Found 38 Solid Tissue Normal files for TCGA-COAD
  Found 100 Primary Tumor files for TCGA-PAAD
  Found 10 Solid Tissue Normal files for TCGA-PAAD

Total files to download: 380
project    sample_type        
TCGA-COAD  Primary Tumor          100
           Solid Tissue Normal     38
TCGA-LUAD  Primary Tumor          100
           Solid Tissue Normal     32
TCGA-PAAD  Primary Tumor          100
           Solid Tissue Normal     10
dtype: int64


In [4]:
# ------------------------------------------------------------------
# Download and merge into a beta-value matrix
# ------------------------------------------------------------------
# Individual sample files are cached by download_methylation_file.
# We also cache the merged matrix — assembling 400+ Series into a
# single DataFrame is slow even when all files are already on disk.

N_SAMPLES = len(file_manifest)
BETA_MATRIX_CACHE = DATA_DIR / f"beta_matrix_{N_SAMPLES}samples.pkl"

if BETA_MATRIX_CACHE.exists():
    beta_matrix = pd.read_pickle(BETA_MATRIX_CACHE)
    print(f"Loaded cached beta matrix ({N_SAMPLES} samples)")
else:
    beta_series = {}
    failed = []

    print("Downloading methylation files...")
    for _, row in tqdm(file_manifest.iterrows(), total=len(file_manifest)):
        try:
            series = download_methylation_file(row['file_id'])
            beta_series[row['file_id']] = series
        except Exception as e:
            failed.append(row['file_id'])
            print(f"  Failed: {row['file_id']} — {e}")

    if failed:
        print(f"\n{len(failed)} files failed to download.")

    beta_matrix = pd.DataFrame(beta_series)
    beta_matrix.to_pickle(BETA_MATRIX_CACHE)
    print(f"Saved merged beta matrix to cache.")

print(f"\nRaw beta matrix shape: {beta_matrix.shape}")
print(f"  {beta_matrix.shape[0]} CpG probes x {beta_matrix.shape[1]} samples")

100%|██████████| 380/380 [12:24<00:00,  1.96s/it]


Saved merged beta matrix to cache.

Raw beta matrix shape: (486426, 380)
  486426 CpG probes x 380 samples


In [5]:
# ------------------------------------------------------------------
# Build metadata table linking file IDs to labels
# ------------------------------------------------------------------

# Create a mapping from file_id to labels
metadata = file_manifest.set_index('file_id')[['project', 'sample_type']].copy()

# Keep only samples we successfully downloaded
metadata = metadata.loc[metadata.index.isin(beta_matrix.columns)]

# Create binary cancer label
metadata['is_cancer'] = (metadata['sample_type'] == 'Primary Tumor').astype(int)

# Create tissue label (cancer type) — only meaningful for tumor samples
metadata['cancer_type'] = metadata['project'].str.replace('TCGA-', '')
metadata.loc[metadata['is_cancer'] == 0, 'cancer_type'] = 'Normal'

print("Sample counts:")
print(metadata.groupby(['cancer_type', 'sample_type']).size())
print(f"\nTotal samples: {len(metadata)}")

Sample counts:
cancer_type  sample_type        
COAD         Primary Tumor          100
LUAD         Primary Tumor          100
Normal       Solid Tissue Normal     80
PAAD         Primary Tumor          100
dtype: int64

Total samples: 380


---
## 2. Preprocessing

Key filtering steps:
- Remove probes with too many missing values
- Remove probes on sex chromosomes (avoid sex-based confounding)
- Remove probes with known SNPs at the CpG site (the exact problem we discussed)
- Impute remaining NAs with the probe median

In [6]:
# ------------------------------------------------------------------
# Preprocessing
# ------------------------------------------------------------------
# Transpose so rows = samples, columns = CpG probes
# This is the standard ML orientation: samples x features

XRAW_CACHE = DATA_DIR / f"X_raw_{N_SAMPLES}samples_missing20.pkl"

if XRAW_CACHE.exists():
    X_raw = pd.read_pickle(XRAW_CACHE)
    print(f"Loaded cached feature matrix ({N_SAMPLES} samples)")
else:
    X_raw = beta_matrix.T
    print(f"Starting shape: {X_raw.shape}")

    # 1. Remove probes with >20% missing values
    missing_frac = X_raw.isnull().mean(axis=0)
    good_probes = missing_frac[missing_frac < 0.2].index
    X_raw = X_raw[good_probes]
    print(f"After missing filter: {X_raw.shape}")

    # 2. Remove sex chromosome probes
    # 450K probe IDs starting with 'cg' are autosomal/X/Y
    # We'll filter by known X/Y probe lists if available,
    # otherwise skip this step and note it as a TODO
    # For now, we keep all probes — in a real analysis you'd
    # cross-reference with the 450K manifest annotation file
    # to remove chrX and chrY probes.
    print("NOTE: Sex chromosome filtering skipped — see TODO below")

    # 3. Impute remaining NAs with column (probe) median
    X_raw = X_raw.fillna(X_raw.median(axis=0))

    X_raw.to_pickle(XRAW_CACHE)
    print(f"Saved feature matrix to cache.")

# 4. Basic sanity check — values should be between 0 and 1
print(f"\nBeta value range: [{X_raw.min().min():.3f}, {X_raw.max().max():.3f}]")
print(f"Final feature matrix: {X_raw.shape[0]} samples x {X_raw.shape[1]} CpG probes")

Starting shape: (380, 486426)
After missing filter: (380, 410462)
NOTE: Sex chromosome filtering skipped — see TODO below
Saved feature matrix to cache.

Beta value range: [0.004, 0.996]
Final feature matrix: 380 samples x 410462 CpG probes


In [ ]:
# ------------------------------------------------------------------
# TODO: Enhanced preprocessing for a more rigorous analysis
# ------------------------------------------------------------------
#
# 1. Download the Illumina 450K manifest from:
#    https://webdata.illumina.com/downloads/productfiles/humanmethylation450/
#    This gives you probe annotations including chromosome, gene, relation
#    to CpG island, and known SNP overlap.
#
# 2. Remove probes on chrX and chrY
# 3. Remove probes flagged with SNPs at the CpG site
# 4. Remove cross-reactive probes (Chen et al. 2013 published a list)
#
# For this starter notebook we skip these for simplicity, but they
# matter for a real analysis and are good talking points in an interview.

---
## 3. Feature Selection

With ~450K features and ~100 samples, we need to reduce dimensionality.
This mirrors GRAIL's panel design step — going from millions of CpGs
down to the most informative subset.

We'll use two approaches:
1. **Univariate filtering** — rank probes by how well they individually distinguish cancer from normal
2. **L1 regularization** — let the model itself select features during training

In [ ]:
# ------------------------------------------------------------------
# Align features and labels
# ------------------------------------------------------------------

# Make sure our X matrix and metadata are aligned
common_samples = X_raw.index.intersection(metadata.index)
X = X_raw.loc[common_samples]
y_cancer = metadata.loc[common_samples, 'is_cancer']
y_type = metadata.loc[common_samples, 'cancer_type']

print(f"Aligned dataset: {X.shape[0]} samples, {X.shape[1]} features")
print(f"\nCancer labels: {y_cancer.value_counts().to_dict()}")
print(f"Type labels: {y_type.value_counts().to_dict()}")

In [ ]:
# ------------------------------------------------------------------
# Train/test split
# ------------------------------------------------------------------
# Stratify by cancer type to ensure balanced representation

X_train, X_test, y_train_cancer, y_test_cancer, y_train_type, y_test_type = \
    train_test_split(
        X, y_cancer, y_type,
        test_size=0.3,
        random_state=42,
        stratify=y_type
    )

print(f"Train: {X_train.shape[0]} samples")
print(f"Test:  {X_test.shape[0]} samples")
print(f"\nTrain cancer type distribution:")
print(y_train_type.value_counts())

In [ ]:
# ------------------------------------------------------------------
# Univariate feature selection (ANOVA F-test)
# ------------------------------------------------------------------
# Select top K most differentially methylated probes between
# cancer and normal. This is a simple proxy for the DMR analysis
# that GRAIL would do with much more sophisticated methods.

K_FEATURES = 5000  # start with top 5000 probes

SELECTOR_CACHE = DATA_DIR / f"feature_selection_{N_SAMPLES}samples_k{K_FEATURES}.pkl"

if SELECTOR_CACHE.exists():
    selected_probes, f_scores = pd.read_pickle(SELECTOR_CACHE)
    X_train_sel = X_train[selected_probes]
    X_test_sel = X_test[selected_probes]
    print(f"Loaded cached feature selection ({len(selected_probes)} probes)")
else:
    selector = SelectKBest(f_classif, k=K_FEATURES)
    selector.fit(X_train, y_train_cancer)

    selected_mask = selector.get_support()
    selected_probes = X_train.columns[selected_mask]

    X_train_sel = X_train[selected_probes]
    X_test_sel = X_test[selected_probes]

    f_scores = pd.Series(selector.scores_, index=X_train.columns)

    pd.to_pickle((selected_probes, f_scores), SELECTOR_CACHE)
    print(f"Saved feature selection to cache.")

top_probes = f_scores.nlargest(20)
print(f"Selected {len(selected_probes)} probes from {X_train.shape[1]}")
print(f"\nTop 20 probes by F-score:")
print(top_probes)

In [ ]:
# ------------------------------------------------------------------
# Visualize top differentially methylated probes
# ------------------------------------------------------------------

fig, axes = plt.subplots(2, 3, figsize=(15, 8))

for idx, probe in enumerate(top_probes.index[:6]):
    ax = axes[idx // 3, idx % 3]
    
    for cancer_type in y_train_type.unique():
        mask = y_train_type == cancer_type
        ax.hist(
            X_train_sel.loc[mask, probe].dropna(),
            bins=20, alpha=0.5, label=cancer_type, density=True
        )
    
    ax.set_title(probe, fontsize=10)
    ax.set_xlabel('Beta value')
    ax.legend(fontsize=8)

plt.suptitle('Top Differentially Methylated Probes: Beta Value Distributions by Type', fontsize=14)
plt.tight_layout()
plt.show()

---
## 4. Cancer Detection Classifier

Binary classification: cancer vs normal.

We'll train:
1. L1-regularized logistic regression (most likely what GRAIL uses)
2. XGBoost for comparison

Key evaluation: **sensitivity at 99.5% specificity** — this is the operating
point GRAIL uses for Galleri, and it's the clinically relevant metric for
a screening test where false positives are costly.

In [ ]:
# ------------------------------------------------------------------
# Logistic Regression with L1 penalty (LASSO)
# ------------------------------------------------------------------

LR_CACHE = DATA_DIR / f"lr_pipeline_{N_SAMPLES}samples_k{K_FEATURES}.joblib"

if LR_CACHE.exists():
    lr_pipeline = joblib.load(LR_CACHE)
    print(f"Loaded cached LR pipeline")
else:
    lr_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegressionCV(
            penalty='l1',
            solver='saga',
            Cs=20,               # test 20 regularization strengths
            cv=StratifiedKFold(5, shuffle=True, random_state=42),
            scoring='roc_auc',
            max_iter=10000,
            random_state=42
        ))
    ])

    print("Training L1 Logistic Regression...")
    lr_pipeline.fit(X_train_sel, y_train_cancer)
    joblib.dump(lr_pipeline, LR_CACHE)
    print(f"Saved LR pipeline to cache.")

# Check how many features the L1 penalty kept
lr_model = lr_pipeline.named_steps['clf']
n_nonzero = np.sum(lr_model.coef_[0] != 0)
print(f"  Non-zero features: {n_nonzero} / {len(selected_probes)}")
print(f"  Best C (regularization): {lr_model.C_[0]:.4f}")

# Predict probabilities on test set
lr_probs = lr_pipeline.predict_proba(X_test_sel)[:, 1]

In [ ]:
# ------------------------------------------------------------------
# XGBoost classifier (for comparison)
# ------------------------------------------------------------------

if HAS_XGB:
    XGB_CACHE = DATA_DIR / f"xgb_pipeline_{N_SAMPLES}samples_k{K_FEATURES}.joblib"

    if XGB_CACHE.exists():
        xgb_pipeline = joblib.load(XGB_CACHE)
        print(f"Loaded cached XGB pipeline")
    else:
        xgb_pipeline = Pipeline([
            ('scaler', StandardScaler()),  # not strictly needed for XGB, but keeps things consistent
            ('clf', XGBClassifier(
                n_estimators=200,
                max_depth=4,
                learning_rate=0.1,
                subsample=0.8,
                colsample_bytree=0.8,
                eval_metric='logloss',
                random_state=42,
                use_label_encoder=False
            ))
        ])

        print("Training XGBoost...")
        xgb_pipeline.fit(X_train_sel, y_train_cancer)
        joblib.dump(xgb_pipeline, XGB_CACHE)
        print(f"Saved XGB pipeline to cache.")

    xgb_probs = xgb_pipeline.predict_proba(X_test_sel)[:, 1]
else:
    print("Skipping XGBoost (not installed)")

In [ ]:
# ------------------------------------------------------------------
# Evaluation: ROC curves and sensitivity at target specificity
# ------------------------------------------------------------------

def sensitivity_at_specificity(y_true, y_score, target_spec=0.995):
    """
    Find sensitivity (recall) at a given specificity threshold.
    This is the key clinical metric — what fraction of cancers
    do we detect while keeping false positive rate at 0.5%?
    """
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    # Specificity = 1 - FPR
    specificities = 1 - fpr
    # Find the threshold closest to target specificity
    idx = np.argmin(np.abs(specificities - target_spec))
    return tpr[idx], specificities[idx], thresholds[idx]


# ROC curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Full ROC
fpr_lr, tpr_lr, _ = roc_curve(y_test_cancer, lr_probs)
auc_lr = roc_auc_score(y_test_cancer, lr_probs)
ax1.plot(fpr_lr, tpr_lr, label=f'L1 LogReg (AUC={auc_lr:.3f})', linewidth=2)

if HAS_XGB:
    fpr_xgb, tpr_xgb, _ = roc_curve(y_test_cancer, xgb_probs)
    auc_xgb = roc_auc_score(y_test_cancer, xgb_probs)
    ax1.plot(fpr_xgb, tpr_xgb, label=f'XGBoost (AUC={auc_xgb:.3f})', linewidth=2)

ax1.plot([0, 1], [0, 1], 'k--', alpha=0.3)
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate (Sensitivity)')
ax1.set_title('ROC Curve — Cancer Detection')
ax1.legend()

# Zoomed ROC at high specificity (the clinically relevant region)
ax2.plot(fpr_lr, tpr_lr, label=f'L1 LogReg', linewidth=2)
if HAS_XGB:
    ax2.plot(fpr_xgb, tpr_xgb, label=f'XGBoost', linewidth=2)

ax2.set_xlim([0, 0.05])  # zoom to FPR < 5%
ax2.axvline(x=0.005, color='red', linestyle='--', alpha=0.5, label='99.5% specificity')
ax2.set_xlabel('False Positive Rate')
ax2.set_ylabel('True Positive Rate (Sensitivity)')
ax2.set_title('ROC Curve — Zoomed to High Specificity')
ax2.legend()

plt.tight_layout()
plt.show()

# Print sensitivity at target specificities
print("\n" + "="*60)
print("SENSITIVITY AT TARGET SPECIFICITY")
print("="*60)

for target_spec in [0.99, 0.995, 0.999]:
    sens_lr, spec_lr, _ = sensitivity_at_specificity(y_test_cancer, lr_probs, target_spec)
    print(f"\nAt {target_spec*100:.1f}% specificity:")
    print(f"  L1 LogReg:  sensitivity = {sens_lr:.3f}")
    if HAS_XGB:
        sens_xgb, spec_xgb, _ = sensitivity_at_specificity(y_test_cancer, xgb_probs, target_spec)
        print(f"  XGBoost:    sensitivity = {sens_xgb:.3f}")

In [ ]:
# ------------------------------------------------------------------
# Inspect model features
# ------------------------------------------------------------------
# One of the advantages of L1 logistic regression: interpretability.
# We can see exactly which CpG probes the model relies on.

coefs = pd.Series(
    lr_model.coef_[0],
    index=selected_probes
)

# Top positive weights (higher methylation → more likely cancer)
top_positive = coefs.nlargest(15)
# Top negative weights (higher methylation → less likely cancer)
top_negative = coefs.nsmallest(15)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

top_positive.plot(kind='barh', ax=ax1, color='firebrick')
ax1.set_title('Top Positive Weights\n(methylation → cancer)')
ax1.set_xlabel('Coefficient')

top_negative.plot(kind='barh', ax=ax2, color='steelblue')
ax2.set_title('Top Negative Weights\n(methylation → normal)')
ax2.set_xlabel('Coefficient')

plt.tight_layout()
plt.show()

print(f"\nTotal non-zero coefficients: {(coefs != 0).sum()}")
print(f"This is analogous to GRAIL's targeted panel — the model is 'using'")
print(f"only {(coefs != 0).sum()} out of {len(coefs)} probes for its prediction.")

---
## 5. Tissue-of-Origin Classifier

Multi-class classification on cancer samples only: LUAD vs COAD vs PAAD.

This is the second stage of the GRAIL pipeline — once you've detected a
cancer signal, predict where it came from.

In [ ]:
# ------------------------------------------------------------------
# Tissue-of-Origin classifier
# ------------------------------------------------------------------

# Filter to cancer samples only
cancer_mask_train = y_train_cancer == 1
cancer_mask_test = y_test_cancer == 1

X_train_cancer = X_train_sel[cancer_mask_train]
X_test_cancer = X_test_sel[cancer_mask_test]
y_train_tof = y_train_type[cancer_mask_train]
y_test_tof = y_test_type[cancer_mask_test]

# Remove 'Normal' from labels (shouldn't be any, but just in case)
assert (y_train_tof == 'Normal').sum() == 0

print(f"Tissue-of-origin training samples: {len(X_train_cancer)}")
print(y_train_tof.value_counts())

TOF_CACHE = DATA_DIR / f"tof_pipeline_{N_SAMPLES}samples_k{K_FEATURES}.joblib"

if TOF_CACHE.exists():
    tof_pipeline = joblib.load(TOF_CACHE)
    print(f"\nLoaded cached ToF pipeline")
else:
    tof_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegressionCV(
            penalty='l1',
            solver='saga',
            multi_class='multinomial',
            Cs=20,
            cv=StratifiedKFold(3, shuffle=True, random_state=42),  # fewer folds due to small sample size
            max_iter=10000,
            random_state=42
        ))
    ])

    print("\nTraining tissue-of-origin classifier...")
    tof_pipeline.fit(X_train_cancer, y_train_tof)
    joblib.dump(tof_pipeline, TOF_CACHE)
    print(f"Saved ToF pipeline to cache.")

tof_preds = tof_pipeline.predict(X_test_cancer)
tof_accuracy = (tof_preds == y_test_tof).mean()

print(f"\nTissue-of-origin accuracy: {tof_accuracy:.1%}")
print(f"(GRAIL reports 93.4% — with 3 cancer types and array data we should do well)")

In [ ]:
# ------------------------------------------------------------------
# Confusion matrix for tissue-of-origin
# ------------------------------------------------------------------

labels = sorted(y_test_tof.unique())
cm = confusion_matrix(y_test_tof, tof_preds, labels=labels)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(
    cm, annot=True, fmt='d', cmap='Blues',
    xticklabels=labels, yticklabels=labels, ax=ax
)
ax.set_xlabel('Predicted Cancer Type')
ax.set_ylabel('True Cancer Type')
ax.set_title('Tissue-of-Origin Confusion Matrix')
plt.tight_layout()
plt.show()

print("\nClassification Report:")
print(classification_report(y_test_tof, tof_preds, labels=labels))

---
## 6. Synthetic Dilution Experiment

This is where we simulate the real ctDNA problem.

In a real blood sample, tumor-derived cfDNA is a tiny fraction of the total.
We simulate this by blending tumor methylation profiles with normal profiles
at known ratios (tumor fractions).

For each tumor fraction, we ask: can our classifier still detect the cancer signal?

**This directly models the concept of limit of detection (LOD).**

In [ ]:
# ------------------------------------------------------------------
# Synthetic plasma mixtures
# ------------------------------------------------------------------

def create_synthetic_plasma(tumor_profile, normal_profile, tumor_fraction):
    """
    Simulate a plasma cfDNA methylation profile by mixing tumor and
    normal profiles at a given tumor fraction.
    
    At the array level, this is equivalent to a weighted average of
    beta values. In real sequencing data, you'd instead be mixing
    reads, but the beta-value average is a reasonable approximation
    for array data.
    
    synthetic_beta = (tumor_fraction * tumor_beta) + ((1 - tumor_fraction) * normal_beta)
    """
    return tumor_fraction * tumor_profile + (1 - tumor_fraction) * normal_profile


# Get our test set tumor and normal samples
test_tumors = X_test_sel[y_test_cancer == 1]
test_normals = X_test_sel[y_test_cancer == 0]

if len(test_normals) == 0:
    print("WARNING: No normal samples in test set — using train normals")
    test_normals = X_train_sel[y_train_cancer == 0]

# Tumor fractions to test (log-spaced from 100% down to 0.1%)
tumor_fractions = [1.0, 0.5, 0.2, 0.1, 0.05, 0.02, 0.01, 0.005, 0.001]

# For each tumor fraction, create synthetic mixtures and test detection
results = []

# Pick a random normal sample to use as the "blood background"
np.random.seed(42)
bg_normal = test_normals.iloc[np.random.randint(len(test_normals))]

print("Creating synthetic plasma mixtures...\n")
for tf in tumor_fractions:
    scores = []
    for i in range(len(test_tumors)):
        tumor = test_tumors.iloc[i]
        synthetic = create_synthetic_plasma(tumor, bg_normal, tf)
        
        # Get cancer probability from our trained classifier
        prob = lr_pipeline.predict_proba(synthetic.values.reshape(1, -1))[0, 1]
        scores.append(prob)
    
    scores = np.array(scores)
    
    # At what threshold would we call "cancer detected"?
    # Use the threshold that gives 99.5% specificity on our test set
    _, _, threshold_995 = sensitivity_at_specificity(y_test_cancer, lr_probs, 0.995)
    detection_rate = (scores >= threshold_995).mean()
    
    results.append({
        'tumor_fraction': tf,
        'mean_score': scores.mean(),
        'detection_rate': detection_rate,
        'n_samples': len(scores)
    })
    
    print(f"  Tumor fraction {tf:>6.1%}: detection rate = {detection_rate:.1%}, "
          f"mean score = {scores.mean():.3f}")

dilution_results = pd.DataFrame(results)

In [ ]:
# ------------------------------------------------------------------
# Plot detection rate vs tumor fraction (LOD curve)
# ------------------------------------------------------------------

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Detection rate
ax1.plot(
    dilution_results['tumor_fraction'] * 100,
    dilution_results['detection_rate'] * 100,
    'o-', linewidth=2, markersize=8, color='firebrick'
)
ax1.set_xscale('log')
ax1.set_xlabel('Tumor Fraction (%)')
ax1.set_ylabel('Detection Rate (%)')
ax1.set_title('Cancer Detection Rate vs Tumor Fraction\n(at 99.5% specificity threshold)')
ax1.axhline(y=50, color='gray', linestyle='--', alpha=0.5, label='50% detection')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Mean classifier score
ax2.plot(
    dilution_results['tumor_fraction'] * 100,
    dilution_results['mean_score'],
    's-', linewidth=2, markersize=8, color='steelblue'
)
ax2.set_xscale('log')
ax2.set_xlabel('Tumor Fraction (%)')
ax2.set_ylabel('Mean Classifier Score')
ax2.set_title('Mean Cancer Probability Score vs Tumor Fraction')
ax2.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5, label='Decision boundary')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Estimate LOD (tumor fraction at 50% detection)
above_50 = dilution_results[dilution_results['detection_rate'] >= 0.5]
if len(above_50) > 0 and len(above_50) < len(dilution_results):
    approx_lod = above_50['tumor_fraction'].min()
    print(f"\nApproximate LOD (50% detection): ~{approx_lod:.1%} tumor fraction")
    print(f"(GRAIL reports LOD of 0.07-0.17% for most tumor types)")
else:
    print("\nCould not estimate LOD from this data.")

---
## 7. Next Steps

This starter notebook covers the basic pipeline. Here are extensions that
would make this a stronger portfolio project and better interview prep:

### Preprocessing improvements
- Download the 450K manifest and filter sex chromosome + SNP-overlapping probes
- Add more cancer types (TCGA has ~33 projects)
- Increase sample counts per cancer type

### Feature engineering
- Instead of individual CpG probes, aggregate into regions (CpG islands, shores, shelves)
- Compute variance-based features (methylation heterogeneity within a region)
- Try PCA or UMAP for dimensionality reduction and visualization

### Modeling
- Compare more model types: Random Forest, SVM, elastic net
- Nested cross-validation for unbiased performance estimation
- Calibration curves — are the predicted probabilities well-calibrated?
- Learning curves — how does performance scale with training data?

### Dilution experiment
- Use multiple different normal backgrounds (not just one)
- Add noise to simulate sequencing variability
- Test LOD separately per cancer type
- Plot sensitivity vs tumor fraction per cancer type (like GRAIL does)

### Clinical framing
- Calculate PPV and NPV at realistic cancer prevalence (0.5-1%)
- Model the impact of different specificity thresholds on screening outcomes
- Compare your panel size (non-zero coefficients) to GRAIL's ~100K regions

In [ ]:
# ------------------------------------------------------------------
# Summary statistics for quick reference
# ------------------------------------------------------------------

print("=" * 60)
print("PROJECT SUMMARY")
print("=" * 60)
print(f"\nDataset: {X.shape[0]} samples, {X.shape[1]} CpG probes")
print(f"Cancer types: {', '.join(PROJECTS)}")
print(f"Features selected: {len(selected_probes)}")
print(f"Features used by L1 model: {n_nonzero}")
print(f"\nCancer Detection (L1 LogReg):")
print(f"  AUC: {auc_lr:.3f}")
for target_spec in [0.99, 0.995]:
    sens, _, _ = sensitivity_at_specificity(y_test_cancer, lr_probs, target_spec)
    print(f"  Sensitivity at {target_spec*100:.1f}% specificity: {sens:.3f}")
print(f"\nTissue-of-Origin Accuracy: {tof_accuracy:.1%}")
print(f"\nDilution results:")
for _, row in dilution_results.iterrows():
    print(f"  {row['tumor_fraction']:>6.1%} tumor fraction → {row['detection_rate']:.0%} detection")